In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install evaluate

In [3]:
from huggingface_hub import login, notebook_login
from transformers import AutoTokenizer
import transformers
import torch
import evaluate

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `AutoGen` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AutoGen`


In [9]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [8]:
# RoBERTa Based Model
model_checkpoint = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer =AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [10]:
def QAinference(model, tokenizer, question, context, device='cpu'):
  inputs = tokenizer(question, context, return_tensors="pt")
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
  #Get the highest probability from the model output for the start and end positions:
  absnswer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  #Decode the predicted tokens to get the answer:
  predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
  answers=tokenizer.decode(predict_answer_tokens)
  print(answers)
  return answers

In [11]:
device='cpu'

question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
inputs = tokenizer(question, context, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
  outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[  2.1531,  -9.2384,  -9.0267,  -9.4839, -10.0052,  -9.6648,  -9.4406,
          -9.8649, -10.0581,  -9.5483,  -9.9120,  -9.7443,  -9.8526,  -5.0742,
          -7.9485,  -8.5879,  -7.8658,  -2.7093,  -7.4798,  -7.3547,  -8.1363,
          -6.1660,  -6.4359,  -6.5302,  -5.6704,   0.3172,  -5.3525,  -4.2764,
          -6.6341,  -4.8675,   7.1324,  -4.3698,  -4.7400,  -4.4959,  -9.6064]]), end_logits=tensor([[ 2.6133, -8.2169, -7.6786, -8.0443, -7.0832, -7.9564, -8.2275, -7.7255,
         -7.0893, -7.5004, -7.2475, -7.5894, -7.4115, -8.3489, -8.2157, -6.1461,
         -8.8072, -3.4159, -6.6166, -6.4676, -8.0165, -8.6780, -8.5398, -7.1475,
         -6.6602,  0.2615, -2.3054, -6.3462, -3.2018, -6.1934,  7.4523, -0.8310,
          2.5671,  0.4864, -7.7582]]), hidden_states=None, attentions=None)

In [12]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [13]:
#Decode the predicted tokens to get the answer
predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
answers=tokenizer.decode(predict_answer_tokens)
print(answers)

 13


In [14]:
device='cpu'
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
answers=QAinference(model, tokenizer, question, context)

 13


In [15]:
metric = evaluate.load("squad")

In [16]:
from transformers import DefaultDataCollator
from torch.utils.data import DataLoader

In [ ]:
!pip install datasets

In [18]:
from datasets import load_dataset

In [19]:
raw_datasets = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [20]:
max_length = 384
stride = 128
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]] #100 questions
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")#100, if no overflow, then sample_map=0-99
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx]) #get example id strings

        sequence_ids = inputs.sequence_ids(i) #[None, 0... None, 1... 1]
        offset = inputs["offset_mapping"][i] #100 size array of tuple (0, 4)
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ] #put None in sequence_id==1, i.e., put questions to None

    inputs["example_id"] = example_ids #string list
    return inputs

In [21]:
valkeyname="validation"
small_eval_set = raw_datasets[valkeyname].select(range(100))

In [22]:
validation_dataset = small_eval_set.map(
            preprocess_validation_examples, #preprocess_function, #preprocess_validation_examples,
            batched=True,
            remove_columns=raw_datasets[valkeyname].column_names,
        )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [23]:
print(len(raw_datasets[valkeyname]))
print(len(validation_dataset))

10570
100


In [24]:
eval_set_for_model = validation_dataset.remove_columns(["example_id", "offset_mapping"])

In [25]:
print(validation_dataset.features.keys())#['input_ids', 'attention_mask', 'offset_mapping', 'example_id']
print(eval_set_for_model.features.keys())#['input_ids', 'attention_mask']

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])
dict_keys(['input_ids', 'attention_mask'])


In [26]:
eval_set_for_model.set_format("torch")

In [27]:
batch_size =32
data_collator = DefaultDataCollator()
eval_dataloader = DataLoader(
  eval_set_for_model, batch_size=batch_size, collate_fn=data_collator
    )

In [28]:
num_val_steps = len(eval_dataloader)
start_logits = []
end_logits = []

In [29]:
from tqdm import tqdm

In [30]:
valprogress_bar = tqdm(range(num_val_steps))
for batch in eval_dataloader:
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
  start_logits.append(outputs.start_logits.cpu().numpy())
  end_logits.append(outputs.end_logits.cpu().numpy())
  valprogress_bar.update(1)

100%|██████████| 4/4 [02:18<00:00, 29.51s/it]

In [31]:
def compute_metrics(start_logits, end_logits, features, examples):
    n_best = 20
    max_answer_length = 30

    #features is after tokenization, examples are original dataset
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [32]:
import numpy as np
import collections

In [33]:
# Evalation with 100 smapels
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
dataset_len=len(validation_dataset)
start_logits = start_logits[: dataset_len]
end_logits = end_logits[: dataset_len]
metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets[valkeyname]
    )
print(metrics)


100%|██████████| 10570/10570 [00:01<00:00, 8821.68it/s]


{'exact_match': 0.9460737937559129, 'f1': 0.9293147722665225}


In [34]:
print(metrics)

{'exact_match': 0.9460737937559129, 'f1': 0.9293147722665225}
